In [1]:
import pandas as pd
from UtilsCreateDataFrame import *
import numpy as np
from dataclasses import dataclass, asdict
import matplotlib.pyplot as plt
import numpy_ext as npe
from statsmodels.tsa.stattools import adfuller
from Strategy import *
from Regression import *
import datetime


In [2]:
path = "BTC_ETH_2021-01.csv.gz"
df_ = pd.read_csv(path)
df_ = df_.set_index(pd.to_datetime(df_.time))
df_.drop(columns = ["time"], inplace = True)
df_

,BTC,ETH
time,,
2021-01-01 00:00:00+00:00,28961.66,737.02
2021-01-01 00:01:00+00:00,29009.91,738.74
2021-01-01 00:02:00+00:00,28989.30,737.82
2021-01-01 00:03:00+00:00,28982.69,737.43
2021-01-01 00:04:00+00:00,28975.65,736.89
...,...,...
2021-01-31 23:55:00+00:00,33215.89,1316.72
2021-01-31 23:56:00+00:00,33103.66,1315.04
2021-01-31 23:57:00+00:00,33069.47,1313.72


In [16]:
# with 1minute frequency a day has 1440 observations
#let's try this on a week 
index_input = df_.index
x, y = df_.to_numpy()[:,0], df_.to_numpy()[:,1]
window = 1440
last = 1440*7
rolling = npe.rolling_apply(linearRegression_np, window, x[:last], y[:last], n_jobs = 4)
df_beta = ResultDataFrame(rolling, index_input[:last])
df_beta

,beta,intercept,r2,res_std,res_mean,stationarity_pvalue
time,,,,,,
2021-01-01 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-01 00:01:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-01 00:02:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-01 00:03:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-01 00:04:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2021-01-07 23:55:00+00:00,0.023285,332.990880,0.826404,10.520993,-1.173185e-13,0.117492
2021-01-07 23:56:00+00:00,0.023275,333.345485,0.826030,10.532041,6.805420e-14,0.112567
2021-01-07 23:57:00+00:00,0.023264,333.709493,0.825598,10.544829,3.615873e-14,0.093566


In [36]:
"""
TEST 1
trading_window = 1 day
"""

result_strategy = applyStrategyRolling(
    df_beta = df_beta,
    df_price = df_,
    trading_window_day = 1, 
    frequency = "1m", 
    thr_pval = 0.1
)
#total return 
tot_ret = 1
for strat in result_strategy[0]:
    for ret in strat.ret_:
        tot_ret *= ret
print("TOT RET:  ", tot_ret)
print("\n\nDECISION DATES")
for dec in result_strategy[1]:
    print(dec, "\n")

TOT RET:   1.2392849230595413


DECISION DATES
beta                   9.511356e-03
intercept              4.517162e+02
r2                     1.762130e-01
res_std                4.590768e+00
res_mean               6.900159e-14
stationarity_pvalue    9.486289e-02
Name: 2021-01-02 11:44:00+00:00, dtype: float64 

beta                  -6.464830e-02
intercept              3.057470e+03
r2                     2.547148e-01
res_std                6.392163e+01
res_mean              -7.124375e-13
stationarity_pvalue    9.625794e-02
Name: 2021-01-04 03:44:00+00:00, dtype: float64 

beta                   3.946798e-02
intercept             -2.234963e+02
r2                     6.467541e-01
res_std                2.645552e+01
res_mean               9.221266e-14
stationarity_pvalue    1.397301e-02
Name: 2021-01-05 03:46:00+00:00, dtype: float64 

beta                   2.529187e-02
intercept              2.327950e+02
r2                     9.031174e-01
res_std                9.882658e+00
res_mean   

In [37]:
"""
TEST 2
trading window = 0.5 day 
"""

result_strategy = applyStrategyRolling(
    df_beta = df_beta,
    df_price = df_,
    trading_window_day = 0.5, 
    frequency = "1m", 
    thr_pval = 0.1
)
#total return 
tot_ret = 1
for strat in result_strategy[0]:
    for ret in strat.ret_:
        tot_ret *= ret
print("TOT RET:  ", tot_ret)
print("\n\nDECISION DATES")
for dec in result_strategy[1]:
    print(dec, "\n")

TOT RET:   1.2947522782969498


DECISION DATES
beta                   9.511356e-03
intercept              4.517162e+02
r2                     1.762130e-01
res_std                4.590768e+00
res_mean               6.900159e-14
stationarity_pvalue    9.486289e-02
Name: 2021-01-02 11:44:00+00:00, dtype: float64 

beta                   1.576827e-02
intercept              2.652390e+02
r2                     9.646952e-01
res_std                3.973742e+00
res_mean               1.221344e-13
stationarity_pvalue    2.071066e-02
Name: 2021-01-02 23:46:00+00:00, dtype: float64 

beta                  -6.464830e-02
intercept              3.057470e+03
r2                     2.547148e-01
res_std                6.392163e+01
res_mean              -7.124375e-13
stationarity_pvalue    9.625794e-02
Name: 2021-01-04 03:44:00+00:00, dtype: float64 

beta                   2.888138e-02
intercept              1.056662e+02
r2                     3.988006e-01
res_std                3.439789e+01
res_mean   

In [41]:
"""
TEST 2
trading window = 0.25 day 
"""

result_strategy = applyStrategyRolling(
    df_beta = df_beta,
    df_price = df_,
    trading_window_day = 0.25, 
    frequency = "1m", 
    thr_pval = 0.1
)
#total return 
tot_ret = 1
for strat in result_strategy[0]:
    for ret in strat.ret_:
        tot_ret *= ret
print("TOT RET:  ", tot_ret)
print("\n\nDECISION DATES")
for dec in result_strategy[1]:
    print(dec, "\n")

TOT RET:   1.1117833509203965


DECISION DATES
beta                   9.511356e-03
intercept              4.517162e+02
r2                     1.762130e-01
res_std                4.590768e+00
res_mean               6.900159e-14
stationarity_pvalue    9.486289e-02
Name: 2021-01-02 11:44:00+00:00, dtype: float64 

beta                   1.573047e-02
intercept              2.669779e+02
r2                     9.440855e-01
res_std                3.770719e+00
res_mean               6.939634e-14
stationarity_pvalue    7.820703e-03
Name: 2021-01-02 17:46:00+00:00, dtype: float64 

beta                   1.577562e-02
intercept              2.650188e+02
r2                     9.646436e-01
res_std                3.978240e+00
res_mean               5.684342e-14
stationarity_pvalue    1.714913e-02
Name: 2021-01-02 23:48:00+00:00, dtype: float64 

beta                   1.467591e-02
intercept              2.984767e+02
r2                     9.454109e-01
res_std                4.556694e+00
res_mean   